# Train / Dev / test sets

> Building a NN is an iterative process

Stractured data is used for Ads, speach, sequirity, logistics.  
Thus coming from NLP, or vision, it is not strigtforward to go to analyzing stractured data. 

Data us usually split into 
- Test set 
- cross-validation set
- test set

Common rule is 70/30 or 60,20,10 for small data sets.  
In case of huge datasets: just a few percent for test sets are sufficient.  

> data from test and cv sets should be coming from **the same distribution**. 

Sometimes only test and cv sets are needed. So overall, generally, 2 datasets are the bare minimum needed. 

#### Bias vs Variance (underfitting vs overfitting)
Consider Train set error and dev set error. (assume that test error __should__ be very low i.e., base error is low)
- If they are $1$ and $11$, this is **high variance**. 
- If they are $11$ and $16$, this is **high bias**. 
- If they are $15$ and $30$, this is **high bias & varaince**. 
- If they are $.5$ and $1$, this is **low bias & varaince**. 

#### Improving high bias / varaince problem

Check the bias (looking for training set).  If bias is high, try other network configurations, ($L$, $\alpha$, ..., architecture)  
**Note** Getting more data here usually _does not_ help

After bias is solved, the high variance problem can be solved by 
(e.g., getting more data, regulariation, more appropriate NN achitecture). 

> Bias/variance tradeoff. Old proplem. Can be solved with Bigger NN (reduces bias) and getting more data (reduces variance)




### Recularzation

> Regularization prevents overfitting.

In, for example, logisitc regression, we are trying to minimze the cost function. In **Regularization** we add the regularization term to the cost
$$
J(w,b) += \frac{\lambda}{2m}b^2
$$
where the notm of $w^2$, the **$L_2$ regularization** term reads
$$
||w^{[l]}||^2 = \sum_{i=1}^{n^{[l]}}\sum_{j=1}^{n^{[l-1]}}( w_{i,j}^{[l]} )^2
$$
One can also consider $L_1$ regularization term $\lambda/m ||w||_1$
Here $w$ is sparse (has too many zeros), and can help with **compressing** the model. 

> $\lambda$, the regularization term, is set using CV set.

In NN, the regularization is added as ter to the cost function where one has to loop over all training examples and units
$$
||w^{[l]}||^2 = \sum_{i=1}^{n^{[l]}}\sum_{j=1}^{n^{[l-1]}}( w_{i,j}^{[l]} )^2
$$
> THis is so-called **Frobinious norm** of the matrix

In the past $dw$ was computed using backprop. Now the update is:
$$
dw^{[l]} = (\text{from backprop}) + \frac{\lambda}{m}w^{[l]};\,\, \frac{\partial J}{\partial w^{[l]}} = dw^{[l]}
$$
> Sometimes $L2$ regularization is called ''weight decay''

### Why regularization reduces overfitting

Regularization term penalizes large wight terms. Thus it reduces the size of the NN (zero out effecto of many hidden units). This reduces the stregth of overfitting. When $\lambda$ is too large, many hidden units effect $\rightarrow 0$. 
For example, for $\tanh()$ becomes linear for small $w$ and the NN becomes linear at large $\alpha$.

### Dropout regularization

> Drop-out reduces overftting

At each training pass, a random set of neurons in layers are turned off. There exist many implemnetations. Consider "Inverted dropout".
For 3rd layer it reads 

$$
\texttt{\text{d3 = np.random.rand(a3.shapep[0],a3.shapep[1])}} \\
a3 = np.multiply(a3,d3) \\
a3 =/ 0.8 = \text{keep probability}
$$

Note that because we shut off some units, we need to reduce the layer output by the "keep probability" factor. 
This assures that the expected value remains the same. 

At test time, no drop-out used. 

### Understanding Drop-out

This is th idea to randomly elliminate the inputs for a given layer. This makes NN less reliant on any particular input. This reduces the size of weights realated to thes einputs and overall shrinks the NN. 

> Drop-out has a similar effect to $L2$ regularization.  

As each layer has diffeent size, for small layers, keep-prop should be small, as too many units can be reduced. For input layer keep-prop is usually 100\%. Overall, this is different  

> Cost function $J$ is no longr _well defined_ when drop-our is used

### Other regularization methods:

- Data augmentation (creating 'new' data by flipping/ropping/resizing/filtering old data)
- Eary stopping (step after a smaller number of iteration utill cost fuction starts rising) This prevents overfitting.

> **Ortogonalization**  When training a NN, we otimize for cost function $J$. Then, there is a separate task of not overfitting, _reducing variance_. 

Note, that early stopping mixes the decreaing $J$ and reducing variacne. 

## Normalazing inputs for faster training

Normalization:
1. Substract the mean $\mu=\frac{1}{m}\sum_{i=1}^m x^{(i)}$: $x:=x-\mu$; 
2. Normalize variance $\sigma^2 = \frac{1}{m}\sum_{i=1}^m (x^{(i)}**2)$, where $**$ is element wise $x^2$; $x /= \sigma$

This makes variance for all inputs $x$ t be $\sigma=1$. 

**Note** test and train sets are normalized with the same $\mu$ and $\sigma$. 

**Normalization** helps speeding up the gradient descend, and it allows to set larger learning rate. 

### Vanishing / Exploding gradients

When training a _very deep_ NN, it is possible that gradients become too small.  
**Solution** carefull choice of random weight initializaton.  

Consider a $L$ NN. and $g(z)$ activation function.  

$y=[...w^{[l]}...] \times x$. 

If each of the $w^{[l]}$ is a bit larger than identity matrix. 
Than 
$$
\hat{y} \propto 
\begin{bmatrix}
1.5 & 0 \\
0, & 1.5
\end{bmatrix}^{[L-1]} \times X
$$
So, if $L$ is large, the $w$ matrix may `explode` due to exponential dependence.   
If weghts are small, however, then the power $L-1$ can make the matrix zero. 

> In a deep NN, activations make explode or get to zero if weights are large or small

### Solving exploding gradient problem

Consider a single Nueron of a NN. $a = g(z)$ and $z=\sum w_i x_i$.  
Note: for larger $n$, the $w_i$ should be smaller to _avoid_ the gradient problem.  
We can set the weights such that the variance of them $\text{var}(w)=1/n$, where $n$ is the number of features that goes into the neuron.  
$$
w^{[l]} = \texttt{\text{np.random.rand(..shape..)}}\times\texttt{\text{np.sqrt}(1/n{[l-1]})}
$$
For $ReLU$ activation function it is a bit different.  

This helps reduce the exploding gradient problem.  
For $\tanh$ activation function, the initialization is different.  
See `Xavier initialization` paper. 

This can eventually be wrapped into a hyperparameter -- how to set variance to 0 for various activation functions. 


### Numerical approximation of gradients

When creating a NN it is usfull to implement *gradient checking*.  
The difrst step in it is to implement *numerically approximate gradients*.  

The numerical approximatrion of gradients is essentially _finite differencing_. Specifically, the two-sided stencil of the finite differencing. 
$$
g(\epsilon) = \lim_{\epsilon\rightarrow0}\frac{f(\theta+\epsilon) - f(\theta-\epsilon)}{2\epsilon} + \mathcal{O}(\epsilon^2)
$$
where the last term is the error. 

For one-sided stencil, usually used in NNs, the error is of the order of $\mathcal{O}(\epsilon)$.  

### Gradient checking

A tool to veryfy the correct implementation of backprop.  

1. Concatenate all $W^{[l]}$ and $b^{[l]}$ after reshaping into a single vector $\theta$. 
2. Do the same for all $dW^{[l]}$ and $W^{[l]}$ to create vector $d\theta$
3. check if $d\theta$ is the slope of the cost function $J(\theta) = J(\theta_1....\theta_n)$. 
This is done via two-sided stencil, i.e.,
$$
d\theta[i]_{\rm approx} = \frac{J(\theta_1...\theta_{i}+\epsilon,...)-J(\theta_1...\theta_{i}-\epsilon,...)}{2\epsilon} = d\theta[i] = \frac{\partial J}{\partial \theta_i}
$$
_for every value of $i$_.  
4. Compare $d\theta_{\rm approx}$ and $d\theta$ via computing _eucledian distance as 
$$
\frac{ || d\theta_{\rm approx} - d\theta||_2 }{||d\theta_{\rm approx}||_2 + ||d\theta||_2} \lesssim 10^{-7} 
$$

where in the last term, it is a criterion. It should not be larger than $10^{-5-7}$. 

### Implementing gradient checking

- This is only for debug. Calculations are very slow.  
- Investigate which component of $d\theta_{\rm approx}[i]$ is far from $d\theta[i]$ 
- Remember regularization (include the regularization term)
- Does not work with dropout (the cost faction $J$ is complex) 
- Check the grad check ar rundom initialization (perhaps again after some training) This is because gradnetns might be incorrectly imlemented but it shows only for very small $W$ $b$.


 


